In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import optimizers
import tensorflow_datasets as tfds

In [16]:
train_data, test_data = tfds.load('horses_or_humans', split=['train', 'test'], as_supervised=True)


In [17]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [18]:
def preprocess_img(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image, label

In [25]:
train_data = train_data.map(preprocess_img)
test_data = test_data.map(preprocess_img)
# Define the batch size
batch_size = 32


In [26]:
# Create the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [27]:
# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [28]:
# Create a new model on top of the pre-trained model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [29]:
# Compile the model
model.compile(optimizer=optimizers.Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
# Train the model
history = model.fit(
    train_data.batch(batch_size),
    steps_per_epoch=info.splits['train'].num_examples // batch_size,
    epochs=10
)

Epoch 1/10
32/32 [==============================] - 54s 2s/step - loss: 1.1575 - accuracy: 0.9629
Epoch 2/10
 1/32 [..............................] - ETA: 6s - loss: 0.0000e+00 - accuracy: 1.0000WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 320 batches). You may need to use the repeat() function when building your dataset.


32/32 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [31]:
# Evaluate the model on the test set
test_score = model.evaluate(test_data.batch(batch_size), steps=test_data.cardinality().numpy() // batch_size)
print(f'Test loss: {test_score[0]}, Test accuracy: {test_score[1]}')

8/8 [==============================] - 13s 2s/step - loss: 0.4929 - accuracy: 0.9883
Test loss: 0.49287235736846924, Test accuracy: 0.98828125
